In [11]:
import numpy as np
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider, FloatSlider

# Generate synthetic stock price data
np.random.seed(42)
days = 100
prices = np.cumsum(np.random.randn(days)) + 100  # Simulate stock prices with random walk
noise = np.random.normal(0, 25, days)  # Add noise to the data
prices_noisy = prices + noise

# Convert data to a tensor for Conv1d
def smooth_prices(kernel_size, w1, w2, w3,stride):
    prices_tensor = torch.tensor(prices_noisy, dtype=torch.float32).unsqueeze(0).unsqueeze(0)

    # Define custom kernel based on weights for different thirds of the window
    weights = np.concatenate([
        np.ones(kernel_size // 3) * w1,  # First third
        np.ones(kernel_size // 3) * w2,  # Middle third
        np.ones(kernel_size // 3) * w3   # Last third
    ])
    weights = weights / np.sum(weights)  # Normalize the weights

    # Apply Conv1d smoothing with the custom kernel
    kernel = torch.tensor(weights, dtype=torch.float32).unsqueeze(0).unsqueeze(0)
    smoothed_prices_tensor = F.conv1d(prices_tensor, kernel, padding=kernel_size // 2, stride=stride)

    # Plot the results
    smoothed_prices = smoothed_prices_tensor.squeeze().numpy()
    plt.figure(figsize=(10, 6))
    plt.plot(prices_noisy, label='Noisy Prices', color='red', alpha=0.5)
    plt.plot(smoothed_prices, label=f'Smoothed Prices (kernel_size={kernel_size})', color='blue')
    plt.title('Effect of Conv1d Smoothing on Time Series')
    plt.xlabel('Days')
    plt.ylabel('Price')
    plt.legend()
    plt.show()

# Interactive sliders to adjust kernel size and weights for the thirds
interact(
    smooth_prices,
    kernel_size=IntSlider(min=3, max=30, step=1, value=15),
    w1=FloatSlider(min=0.1, max=1.0, step=0.1, value=0.1, description='Weight 1st Third'),
    w2=FloatSlider(min=0.1, max=1.0, step=0.1, value=0.2, description='Weight 2nd Third'),
    w3=FloatSlider(min=0.1, max=1.0, step=0.1, value=0.7, description='Weight 3rd Third'),
    stride=IntSlider(min=1, max=15, step=1, value=1, description='stride')
)

interactive(children=(IntSlider(value=15, description='kernel_size', max=30, min=3), FloatSlider(value=0.1, de…

<function __main__.smooth_prices(kernel_size, w1, w2, w3, stride)>